In [7]:
!pip install keras.tuner

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 127.9/127.9 kB 3.8 MB/s eta 0:00:00
     -------------------------------------- 950.8/950.8 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 239.8/239.8 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 101.3/101.3 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 3.7 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
print(tf.__version__)

2.14.0


In [8]:
fashion_mnist = keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [10]:
train_images = train_images/255.0
test_images = test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images =train_images.reshape(len(train_images),28,28,1)
test_images =test_images.reshape(len(test_images),28,28,1)

In [56]:
def build_model(hp):
    model = keras.Sequential([keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=120,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),activation='relu',input_shape=[28,28,1]),
                              keras.layers.Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),activation='relu'),
                              keras.layers.Flatten(),
                              keras.layers.Dense(units=hp.Int('dense_1_units',min_value=32,max_value=120,step=16),activation='relu'),
                              keras.layers.Dense(10,activation='softmax')])
                              
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [57]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [58]:
tuner_search = RandomSearch(build_model,
                           objective='val_accuracy',
                           max_trials=5,
                           directory='output',
                           project_name="Mnist Fashion")

Reloading Tuner from output\Mnist Fashion\tuner0.json


In [59]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 08m 28s]
val_accuracy: 0.9086666703224182

Best val_accuracy So Far: 0.9136666655540466
Total elapsed time: 00h 37m 35s


In [62]:
model=tuner_search.get_best_models(num_models=1)[0]

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        27680     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 96)                1769568   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1799178 (6.86 MB)
Trainable params: 1799178 (6.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 183s 108ms/step - loss: 0.1274 - accuracy: 0.9525 - val_loss: 0.2613 - val_accuracy: 0.9122
Epoch 5/10
1688/1688 [==============================] - 179s 106ms/step - loss: 0.0859 - accuracy: 0.9678 - val_loss: 0.3183 - val_accuracy: 0.9087
Epoch 6/10
1688/1688 [==============================] - 172s 102ms/step - loss: 0.0617 - accuracy: 0.9777 - val_loss: 0.3550 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 189s 112ms/step - loss: 0.0451 - accuracy: 0.9841 - val_loss: 0.4234 - val_accuracy: 0.9070
Epoch 8/10
1688/1688 [==============================] - 172s 102ms/step - loss: 0.0304 - accuracy: 0.9894 - val_loss: 0.4283 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 165s 98ms/step - loss: 0.0244 - accuracy: 0.9910 - val_loss: 0.5030 - val_accuracy: 0.9128
Epoch 10/10
1688/1688 [==============================] - 180s 106ms/step - loss: 0.0236 - accuracy: 0.9921 - val_